First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import cv2

In [2]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [3]:
model.load_state_dict(torch.load('road_following_model.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [4]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [5]:
torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

Load the optimized model by executing the cell below

In [2]:
import ipywidgets
from IPython.display import display
import traitlets
import time
from utils import preprocess
import numpy as np

In [3]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

<All keys matched successfully>

Create the racecar class

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [6]:
class ApexPrediction(traitlets.HasTraits):
    x = traitlets.Float(default_value=0.0)
    y = traitlets.Float(default_value=0.0)
    
apex = ApexPrediction()

In [7]:
def PID(Kp, Ki, Kd, MV_bar=0):
    # initialize stored data
    e_prev = 0
    t_prev = -100
    I = 0
    
    # initial control
    MV = MV_bar
    
    while True:
        # yield MV, wait for new t, PV, SP
        t, PV, SP = yield MV
        
        # PID calculations
        e = SP - PV
        
        P = Kp*e
        I = I + Ki*e*(t - t_prev)
        D = Kd*(e - e_prev)/(t - t_prev)
        
        MV = MV_bar + P + I + D
        
        # update stored data for next iteration
        e_prev = e
        t_prev = t

In [8]:
apex_widget = ipywidgets.FloatSlider(min=-1.0, max=1.0, value=0.0, description='apex x')
steering_widget = ipywidgets.FloatSlider(min=-1.0, max=1.0, value=0.0, description='steering')

In [9]:
traitlets.dlink((apex, 'x'), (apex_widget, 'value'))
traitlets.dlink((car, 'steering'), (steering_widget, 'value'))

In [10]:
display(apex_widget, steering_widget)

FloatSlider(value=0.0, description='apex x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [ ]:
KP = 0.6
KI = 0.0
KD = 0.0
STEERING_BIAS = 0.00

car.throttle = -0.05

controller = PID(KP, KI, KD)       # create pid control
controller.send(None)              # initialize

t = time.time()

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    
    apex.x = float(output[0]) #x
    
    MV = controller.send([time, apex.x, 0.0]) + STEERING_BIAS
    
    if MV > 1.0:
        MV = 1.0
    elif MV < -1.0:
        MV = -1.0
        
    car.steering = MV
    
    t = time.time()

Exiting... 
Cleaning up pins
